In [ ]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
from model.Lyapunov_Worm_deconstruction import Lyapunov_Worm_Deconstruction
from Bandit.Expected_Value import CalculateMeanValue
from Bandit.BanditGame import Play
from tqdm import tqdm
from multiprocessing import  Process


dim = 2
varargin = {
    'step_num': 400,  # how many steps to run the brain circuit before executing the next movement
    'tau': np.ones(dim),  # decay time constant
    'weights_in': np.ones(dim) * 1.,  # input weights
    'rs': np.ones(dim) * .5,  #
    'w': np.ones(dim) * 4,  # weight of mutual inhibition
    'k': 7. * np.ones(dim),  # sigmoid center
    'n': 1.5 * np.ones(dim),  # sigmoid slope
    'bi': np.ones(dim) * 5.7,  # baseline production
    'dt': 1.2,  # size of timesteps
    'nsf': 1.25,  # noise level
    'w_avg_comp': 1. / 20.,
    'w_std_comp': 1. / 4. * 1.
}
# seed = 1
# np.random.seed(seed)
conds = ['RR','SS','RS','SR']
bandit_std_dict = {
    'RR': [4., 4.],
    'SS': [0., 0.],
    'RS': [4., 0.],
    'SR': [0.,4.]
    
}
def Play_bandit(cond):
    bandit_std_list = bandit_std_dict[cond]
    prior = None
    raw_path = f'bandit_data/raw_{dim}D_fan_'+cond+'high1.csv'
    analysis_path = f'bandit_data/analysed_{dim}D_fan_'+cond+'high1.csv'
    p = Play(varargin=varargin, dim=dim, force_times=2, prior=prior, init=True, bandit_mean=0, mean_std=10, bandit_std_list=bandit_std_list, subject=1, block=20000, trial=20, save=True, save_path=raw_path, save_mode='w')
    p.play()
    k = CalculateMeanValue(raw_path, analysis_path, trial=20, dim=dim, prior=prior)
    k.run()
    
if __name__ == '__main__':
    process_list = []
    for cond in conds:
        p = Process(target=Play_bandit,args=(cond,))
        p.start()
        process_list.append(p)
    
    for i in process_list:
        p.join()